In [1]:
import codecs, nltk, string, os, gensim
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


exclude = set(string.punctuation)

# this represent any text as a single "doc-embedding" we use it both for the query and the sentences
# input should be a string
def text_embedding(text):
    
    #you should check in the embeddings you use if the words have been lowercased or not. 
    #try ask the embedding for "barack" and for "Barack"
    # if the Barack works, then comment the following line
    text = text.lower()
    
    # we tokenize the text in single words
    text = nltk.tokenize.WordPunctTokenizer().tokenize(text)
    
    # we remove numbers and punctuation
    text = [token for token in text if token not in exclude and token.isalpha()]
    
    
    doc_embed = []
    
    # for each word we get the embedding and we append it to a list
    for word in text:
            try:
                embed_word = emb_model[word]
                doc_embed.append(embed_word)
            except KeyError:
                continue
    # we average the embeddings of all the words, getting an overall doc embedding
    if len(doc_embed)>0:
        avg = [float(sum(col))/len(col) for col in zip(*doc_embed)]

        avg = np.array(avg).reshape(1, -1)

        # the output is a doc-embedding
        return avg
    else:
        return "Empty"

C:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#import here your word-embeddings - put the path to the file (if it's .bin change the binary to True)
#emb_model = gensim.models.KeyedVectors.load_word2vec_format('../../resources/small-embeddings.txt', binary=False)

# german wikipedia from https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md
#emb_model = gensim.models.KeyedVectors.load_word2vec_format('/Users/federiconanni/Downloads/wiki.de.vec', binary=False)
emb_model = gensim.models.KeyedVectors.load_word2vec_format('C:/Users/Dr. J/Desktop/wiki.de.vec', binary=False)

# switch for in-domain 
#emb_model = gensim.models.KeyedVectors.load_word2vec_format('C:/QTA/topic-detect/in-domain-embeddings.txt', binary=False)



In [5]:
#this can be a list of words on the same fine-grained topic, like "people", "elites"
# add more words after a space to make it more precise
# query = "volk bürger"

#focus on "key key" terms based on dictionary, let the embeddings find more 
#query = "volk"
#query = "bürger einbürgerung steuerzahler gemeinschaft"
#query = "elite politiker establishment herrschend"
#query = "korruption täuschung betrügen verrat schämen skandal wahrheit unehrlich lüge"
#query = "verantwortung glaubwürdigkeit"
#query = "souverän neutral"
#query = "demokratisch referendum volksabstimmung volksinitiative"
#query = "konsens kompromiss"
query = "repräsentation parlament regierung "
#query = "populisten populismus demagogisch demagogen"

# populism at its best?
#query = "volk elite souverän"

# efficient flip-side query?
#query = "konsens repräsentation populismus"

# query = "volk elite souverän parlament regierung repräsentation"

query_emb = text_embedding(query)

# add the path to the folder where you have your manifestos as text documents
# collection_path = "C:/Users/Dr. J/Dropbox/sparserhetoric/deu2017/"

# only Germany for less output 
#collection_path = "C:/Users/Dr. J/Dropbox/sparserhetoric/germany17/"

# German texts last three elections for Germany, Austria and Switzerland - 63 manifestos from polidoc.net
collection_path = "C:/Users/Dr. J/Dropbox/sparserhetoric/polidoc_allgerman/"

In [6]:
# this will be a dictionary of documents, for example manifestos, divided in sentences, which are represented as sentence embeddings
collection = {}

# you loop over the folder
for filename in os.listdir(collection_path):
    # you open each file
    # note encoding 
    content = codecs.open(collection_path+filename,"r","utf-8").read()
    # you split it in sentences
    content = nltk.sent_tokenize(content)
    
    # you represent each sentence in each document as a word-embedding, which captures the meaning of the sentence
    content = [[sent, text_embedding(sent)] for sent in content if type(text_embedding(sent))!= str]
    collection[filename] = content

In [7]:
# now, the information retrieval part

for filename,sentences in collection.items():
    
    # compare the cosine similarity between the embedding of the query and each sentence embedding
    ranking = [[sent, cosine_similarity(query_emb,sent_emb)[0][0]] for sent, sent_emb in sentences]
    # you rank them, based on the similarity
    ranking.sort(key=lambda x: x[1],reverse=True)
    
    print (filename)
    # you can change here for having more sentences as output
   # for sent, score in ranking[:20]:
    for sent, score in ranking[:3]:
        print (sent, score)
    print (" \n")

41113.000.2009.1.1.txt
Deshalb wollen wir das Europäischen Parlament
gegenüber der Kommission und dem Ministerrat weiter stärken – in der Außen- und Sicherheitspolitik, in der Sozialpolitik, in der Haushalts- und Steuerpolitik.Wir wollen die Bundesregierung bei der Gesetzgebung des Europäischen Rates endlich unter eine wirksame Kontrolle
des Bundestages stellen. 0.7060575552652175
 … stärkt Parlamente und schafft mehr Transparenz in der Politik. 0.6802855373663416
Starker und transparenter Parlamentarismus
Demokratie braucht starke und transparente Parlamente. 0.6789471854533655
 

41113.000.2013.1.1.txt
Parlamente stärken, Parteien öffnen
Die repräsentative Demokratie ist Garant für die Vertretung der gesamten Gesellschaft gegenüber Einzelinteressen. 0.720931813549303
Stattdessen sollen unter dem Vorsitz des Europäischen Parlaments Vertreterinnen und Vertreter der EU-Kommission, der nationalen Parlamente und Regierungen sowie der Zivilgesellschaft und SozialpartnerInnen zusammentrete

41521.000.2013.1.1.txt
Seit dem
Inkrafttreten des Musterabkommens hat die unionsgeführte Bundesregierung 42 solcher Abkommen
mit anderen Staaten unterzeichnet – 36 allein in dieser Wahlperiode. 0.6455068008565912
Polizisten als Garanten der inneren Sicherheit unterstützen – ihren Schutz weiter verbessern
CDU und CSU haben Vertrauen in Polizei und Justiz. 0.6219029853083876
Zudem setzen wir auf ausgeglichene Haushalte in allen EU-Mitgliedstaaten und verteidigen die
Unabhängigkeit der Europäischen Zentralbank, die nach dem Vorbild der Deutschen Bundesbank
von uns durchgesetzt wurde. 0.6214644238357965
 

41521.000.2017.1.1.txt
Für den vernetzten Ansatz wollen wir eine zentrale Koordinierung
innerhalb der Bundesregierung und im Parlament schaffen. 0.6525187889814605
CDU und CSU bekräftigen die Rentenreform der Großen Koalition von 2007. 0.641895172917619
Wir werden künftig
einen Sonderbeauftragten der Bundesregierung für weltweite Religionsfreiheit
berufen. 0.6359683744267077
 

41523.000

42420.000.2013.1.1.txt
Deshalb fordern wir:
-	Ausstieg aus dem ESM- und EFSF-Haftungswahnsinn zum Schutz des Volksvermögens in Österreich
-	Nein zum von der EU betriebenen gemeinsamen Schuldenmachen der reichen und schwachen Länder
-	Senkung der österreichischen EU-Beiträge nach dem Muster von Großbritannien
-	Persönliche Haftung und strafrechtliche Konsequenzen für Pleitemanager von Spekulationsbanken
-	Schaffung der Möglichkeit von Bankkonkursen und Stärkung der Kontrolle der Bankgeschäfte
 

Politik der „Nächstenliebe“ heißt:
Direkte Demokratie ohne Wenn und Aber
Laut Bundesverfassung ist das österreichische Volk der Souverän. 0.6302338824177004
Deshalb fordern wir:
-	Verbindliche Volksabstimmung, wenn ein Volksbegehren von mehr als 250.000 Wahlberechtigten unterstützt wird
-	Die Möglichkeit, ein Gesetz über eine Veto-Volksabstimmung außer Kraft zu setzen
-	Erleichterung der Durchführung von Volksbefragungen
-	Leichterer Zugang bei der Ausübung der direkten Demokratie (z.B. 0.6

42520.000.2017.1.1.txt
Um die Legitimation der Regierung zu stärken und das Parlament aufzuwerten, schlagen wir zudem vor, dass die Mitglieder einer neuen
Regierung vor der Angelobung ein
Hearing durch die Abgeordneten zum Nationalrat durchlaufen sollen. 0.7466862313356749
Dem Referendum für die Verfassungsänderung haben beispielsweise 73 % der Wahlberechtigten zugestimmt. 0.6866195803943123
Unsere
Maßnahmen

Die Instrumente
der direkten Demokratie stärken, indem wir vermehrt
Volksbefragungen
und Volksabstimmungen
durchführen

1- 2 fixe Tage pro Jahr, an denen Anliegen zur Abstimmung oder Befragung gebracht werden können

Volksabstimmung bei Volksbegehren, das von mehr
als 10 % der Bevölkerung unterschrieben wird

Hearing für neue Regierungsmitglieder vor der Entwicklung der Wahlbeteiligung an Nationalratswahlen
Ernennung durch die Abgeordneten zum Nationalrat

Vorzugsstimmen stärker berücksichtigen bei der Vergabe von Mandaten

Zurück
an die Spitze. 0.6794842525162297
 

42951.000.200

43520.000.2015.1.1.txt
Keine andere Partei setzt sich im Parlament häufiger durch. 0.6827142264983751
Für ein funktioniere Asylwesen
Die Asylgesetzrevision wurde im Parlament massgeblich von der CVP geprägt. 0.6533405788619611
Im Parlament ist die CVP sehr erfolgreich. 0.6430547854825948
 

43530.000.2007.1.1.txt
﻿Wahlplattform 2007 – 2011
der Evangelischen Volkspartei der
Schweiz (EVP)
Eine lebenswerte Schweiz sichern durch... ... gestärkte Familien
- Kindergeld statt Subventionswirrwarr gibt Eltern freie Wahl beim Teilen von Erziehungs- und Berufsarbeit. 0.6147735082295394
Die EVP tritt ein für die konsequente Umsetzung des Kartellgesetzes, für Parallelimporte aus dem EU-Raum und den Abbau der entsprechenden Handelshemmnisse. 0.583195050594826
Auf Bundesebene wird eine vernünftig ausgestaltete Erbschafts- und Schenkungssteuer zur Mitfinanzierung der Sozialversicherungen eingeführt. 0.5591912203385119
 

43530.000.2011.1.1.txt
Grenzen der Religionsfreiheit: Erstens Verfassung und Gese